In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Start SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# Load in data from S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [4]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDIJS7QYB6XNR|          5|            0|          0|   N|                Y|
|R36ED1U38IELG8|          5|            0|          0|   N|                Y|
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3BPETL222LMIM|          5|            0|          0|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|                Y|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|                Y|
|R1CB783I7B0U52|          1|            0|          1|   N|     

In [5]:
# Filter the data and create a new DataFrame or table to retrieve all the rows
# where the total_votes count is equal to or greater than 20 to pick reviews that
# are more likely to be helpful and to avoid having division by zero errors later on.

total_votes_20_df = vine_df.filter('total_votes>=20')
total_votes_20_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|ROP6ITXO8K5V2 |5          |23           |27         |N   |Y                |
|R3ND1LVU7AXCVF|1          |21           |21         |N   |Y                |
|R9I5FOLKU99RY |5          |19           |20         |Y   |N                |
|R1QS8AOD6HX3ED|4          |59           |81         |N   |N                |
|R3ED60RC69CCQ6|5          |22           |23         |Y   |N                |
|R2MDI1TP46VSYD|3          |9            |21         |N   |Y                |
|R2JM687C525WR9|3          |33           |33         |Y   |N                |
|R2LWX4TZ67FWPT|4          |50           |50         |N   |N                |
|R29IYHPYD14AGI|3          |84           |84         |N   |Y                |
|R1F2I723WRK5QV|5          |20           |20         |N   |Y    

In [6]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame
# or table to retrieve all the rows where the number of helpful_votes divided by
# total_votes is equal to or greater than 50%.

import pyspark.sql.functions as Fnc
helpful_total_df = total_votes_20_df.withColumn('helpful_pct', Fnc.round((total_votes_20_df.helpful_votes/total_votes_20_df.total_votes)*100,5))
cleaned_pct_df = helpful_total_df.filter('helpful_pct >= 50')
cleaned_pct_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_pct|
+--------------+-----------+-------------+-----------+----+-----------------+-----------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|   85.18519|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|      100.0|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|       95.0|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|   72.83951|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|   95.65217|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|      100.0|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|      100.0|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|      100.0|
|R1F2I723W

In [7]:
cleaned_pct_df.count()

63294

In [8]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or
# table that retrieves all the rows where a review was written as part of the
# Vine program (paid), vine == 'Y'.

paid_vine_df = cleaned_pct_df.filter("vine =='Y'")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_pct|
+--------------+-----------+-------------+-----------+----+-----------------+-----------+
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|       95.0|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|   95.65217|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|      100.0|
| RUB4AUWGHG16G|          4|           25|         27|   Y|                N|   92.59259|
|R1S3IDYJUA6V2G|          5|           28|         32|   Y|                N|       87.5|
| RZFLN8FD5Y2M6|          4|           25|         28|   Y|                N|   89.28571|
|R1V8NDXD9SETO5|          3|          320|        352|   Y|                N|   90.90909|
|R2H4VEFWHP50M3|          5|           26|         36|   Y|                N|   72.22222|
|R3DP6Y0A8

In [9]:
paid_vine_df.count()

1266

In [10]:
# Repeat Step 3, but this time retrieve all the rows where the review was not
# part of the Vine program (unpaid), vine == 'N'.

unpaid_vine_df = cleaned_pct_df.filter("vine =='N'")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_pct|
+--------------+-----------+-------------+-----------+----+-----------------+-----------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|   85.18519|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|      100.0|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|   72.83951|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|      100.0|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|      100.0|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|      100.0|
| RRO8C1IVZMD6H|          5|           22|         22|   N|                N|      100.0|
|R2GHOWIWDVLN18|          1|           28|         36|   N|                Y|   77.77778|
|R30PJ9XUH

In [11]:
unpaid_vine_df.count()

62028

In [12]:
# Determine the total number of reviews

total_reviews = cleaned_pct_df.count()
total_reviews


63294

In [13]:
# Determine the number of 5-star reviews

total_5_star = cleaned_pct_df.filter('star_rating == 5').count()
total_5_star

30414

In [14]:
# Determine the number of reviews below 5-star

total_below5 = cleaned_pct_df.filter('star_rating < 5').count()
total_below5

32880

In [15]:
# Determine the number of paid reviews below 5-star

total_below5_paid = paid_vine_df.filter('star_rating < 5').count()
total_below5_paid

834

In [16]:
# Determine the number of 5-star paid reviews

total_5_star_paid = paid_vine_df.filter('star_rating == 5').count()
total_5_star_paid

432

In [17]:
# Determine the number of 5-star unpaid reviews

total_5_star_unpaid = unpaid_vine_df.filter('star_rating == 5').count()
total_5_star_unpaid

29982

In [18]:
# Determine the number of unpaid reviews below 5-star

total_below5_unpaid = unpaid_vine_df.filter('star_rating < 5').count()
total_below5_unpaid

32046

In [19]:
# Determine the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [20]:
# Find total 5 star review percentage for cleaned dataframe
pct_5_star = round((total_5_star / total_reviews)*100,5)
print(f'Percent of all reviews that were 5 star: {pct_5_star}%')

Percent of all reviews that were 5 star: 48.05195%


In [21]:
# Find total 5 star paid reviews and divide by total 5 star reviews
pct_5_star_paid = round((total_5_star_paid / paid_vine_df.count())*100,5)
print(f'Percent of Paid Vine reviews that were 5 star: {pct_5_star_paid}%')

Percent of Paid Vine reviews that were 5 star: 34.12322%


In [22]:
# Find total 5 star unpaid reviews and divide by total 5 star reviews
pct_5_star_unpaid = round((total_5_star_unpaid / unpaid_vine_df.count())*100,5)
print(f'Percent of Unpaid non-Vine reviews that were 5 star: {pct_5_star_unpaid}%')

Percent of Unpaid non-Vine reviews that were 5 star: 48.33624%
